In [1]:
import os
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import udf

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1568855464994_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#s3 locations for inputs and outputs
input_data = "s3a://dend-capstone-project-donors-choose/"
output_data = "s3a://dend-capstone-project-donors-choose/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Teachers

In [3]:
teachers_data = input_data + 'Teachers.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_teachers = spark.read.csv(teachers_data, inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
columns = ['Teacher ID', 'Teacher Prefix', 'Teacher First Project Posted Date']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
new_columns = [column.replace(" ", "_").lower() for column in columns]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#update column names, drop duplicates
df_teachers = df_teachers.select(*columns).dropDuplicates().toDF(*new_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#change data type to date
df_teachers = df_teachers.withColumn('teacher_first_project_posted_date', F.to_date('teacher_first_project_posted_date'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#fill the blank values
df_teachers = df_teachers.fillna({'teacher_prefix': 'NA'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_teachers.printSchema()
df_teachers.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- teacher_id: string (nullable = true)
 |-- teacher_prefix: string (nullable = false)
 |-- teacher_first_project_posted_date: date (nullable = true)

+--------------------+--------------+---------------------------------+
|          teacher_id|teacher_prefix|teacher_first_project_posted_date|
+--------------------+--------------+---------------------------------+
|001e2dde269f90cbc...|          Mrs.|                       2018-04-23|
+--------------------+--------------+---------------------------------+
only showing top 1 row

In [11]:
df_teachers.createOrReplaceTempView('teachers')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#data quality check
spark.sql('''select count(*) from teachers''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|  402900|
+--------+

In [13]:
df_teachers.write.format("csv").save(output_data + 'teachers')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Donors

In [14]:
donors_data = input_data + 'Donors.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_donors = spark.read.csv(donors_data, inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
columns = ['Donor ID', 'Donor City', 'Donor State', 'Donor Is Teacher']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
new_columns = [column.replace(" ", "_").lower() for column in columns]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#update column names, drop duplicates
df_donors = df_donors.select(*columns).dropDuplicates().toDF(*new_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#fill the blank values
df_donors = df_donors.fillna({'donor_city': 'NA'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_donors.printSchema()
df_donors.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- donor_id: string (nullable = true)
 |-- donor_city: string (nullable = false)
 |-- donor_state: string (nullable = true)
 |-- donor_is_teacher: string (nullable = true)

+--------------------+----------+-----------+----------------+
|            donor_id|donor_city|donor_state|donor_is_teacher|
+--------------------+----------+-----------+----------------+
|0001b46c36c7c900c...|   Chicago|   Illinois|              No|
+--------------------+----------+-----------+----------------+
only showing top 1 row

In [21]:
df_donors.createOrReplaceTempView('donors')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#data quality check
spark.sql('''select count(*) from donors''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
| 2122640|
+--------+

In [23]:
df_donors.write.format("csv").save(output_data + 'donors')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Schools

In [24]:
schools_data = input_data + 'Schools.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_schools = spark.read.json(schools_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
columns = ['school_id', 'school_name','school_state', 'school_city', 'school_metro_type', 'school_percentage_free_lunch']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#update column names, drop duplicates
df_schools = df_schools.select(*columns).filter(df_schools.school_id != 'null').dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#fill the blank values
df_schools = df_schools.fillna({'school_city': 'NA'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
to_int = F.udf(lambda x: int(x) if x else None, T.IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#change data type to integer
df_schools = df_schools.withColumn('school_percentage_free_lunch',  to_int('school_percentage_free_lunch'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#fill the blank values
df_schools = df_schools.fillna({'school_percentage_free_lunch': 0})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
percentage = 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#categorize free lunch level
def freeLunchLevel(percentage):
    if percentage>0 and percentage <=30: return 'low'
    elif percentage>30 and percentage <=60: return 'medium'
    elif percentage>60 and percentage <=100: return 'high'
    else: return 'NA'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#UDF
udfFreeLunchLevel=udf(freeLunchLevel, T.StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_schools = df_schools.withColumn('free_lunch_level', udfFreeLunchLevel('school_percentage_free_lunch'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_schools.printSchema()
df_schools.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- school_id: string (nullable = true)
 |-- school_name: string (nullable = true)
 |-- school_state: string (nullable = true)
 |-- school_city: string (nullable = false)
 |-- school_metro_type: string (nullable = true)
 |-- school_percentage_free_lunch: integer (nullable = false)
 |-- free_lunch_level: string (nullable = true)

+--------------------+--------------------+------------+-----------+-----------------+----------------------------+----------------+
|           school_id|         school_name|school_state|school_city|school_metro_type|school_percentage_free_lunch|free_lunch_level|
+--------------------+--------------------+------------+-----------+-----------------+----------------------------+----------------+
|01726bdb4bb26e34d...|North Elementary ...|   Tennessee|   Altamont|            rural|                          83|            high|
+--------------------+--------------------+------------+-----------+-----------------+----------------------------+----------------

In [37]:
df_schools.createOrReplaceTempView('schools')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#data quality check
spark.sql('''select count(*) from schools''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|   72993|
+--------+

In [39]:
df_schools.write.format("csv").save(output_data + 'schools')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Donations

In [40]:
donations_data = input_data + 'Donations.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_donations = spark.read.csv(donations_data, inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
columns = ['Project ID', 'Donation ID', 'Donor ID', 'Donation Included Optional Donation', 'Donation Amount', 'Donation Received Date']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
new_columns = [column.replace(" ", "_").lower() for column in columns]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#update column names, drop duplicates
df_donations = df_donations.select(*columns).dropDuplicates().toDF(*new_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#change data type to date
df_donations = df_donations.withColumn('donation_received_date', F.to_date('donation_received_date'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#extract year
df_donations = df_donations.withColumn('year', F.year('donation_received_date'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#filter only donations made in 2018 fpr analysis purpose
df_donations = df_donations.where(df_donations['Year'] == 2018)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_donations.printSchema()
df_donations.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- project_id: string (nullable = true)
 |-- donation_id: string (nullable = true)
 |-- donor_id: string (nullable = true)
 |-- donation_included_optional_donation: string (nullable = true)
 |-- donation_amount: double (nullable = true)
 |-- donation_received_date: date (nullable = true)
 |-- year: integer (nullable = true)

+--------------------+--------------------+--------------------+-----------------------------------+---------------+----------------------+----+
|          project_id|         donation_id|            donor_id|donation_included_optional_donation|donation_amount|donation_received_date|year|
+--------------------+--------------------+--------------------+-----------------------------------+---------------+----------------------+----+
|5600beb7e62d3d5fc...|39b19a12c419a43ff...|1430bc246a0f7f2b2...|                                Yes|         186.64|            2018-04-25|2018|
+--------------------+--------------------+--------------------+----------------------

## Projects

In [49]:
projects_data = input_data + 'Projects_2018.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_projects = spark.read.csv(projects_data, inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
columns = ['Project ID', 'School ID', 'Teacher ID', 'Project Type', 'Project Grade Level Category', 'Project Cost', 'Project Posted Date', 'Project Expiration Date', 'Project Fully Funded Date']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
new_columns = [column.replace(" ", "_").lower() for column in columns]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
#update column names, drop duplicates
df_projects = df_projects.select(*columns).dropDuplicates().toDF(*new_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
#fill the blank values
df_projects = df_projects.fillna({'project_fully_funded_date': 'NA'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
#change data type to date
df_projects = df_projects.withColumn('project_posted_date', F.to_date('project_posted_date')).withColumn('project_expiration_date', F.to_date('project_expiration_date')).withColumn('project_fully_funded_date', F.to_date('project_fully_funded_date'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
df_projects.printSchema()
df_projects.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- project_id: string (nullable = true)
 |-- school_id: string (nullable = true)
 |-- teacher_id: string (nullable = true)
 |-- project_type: string (nullable = true)
 |-- project_grade_level_category: string (nullable = true)
 |-- project_cost: double (nullable = true)
 |-- project_posted_date: date (nullable = true)
 |-- project_expiration_date: date (nullable = true)
 |-- project_fully_funded_date: date (nullable = true)

+--------------------+--------------------+--------------------+------------+----------------------------+------------+-------------------+-----------------------+-------------------------+
|          project_id|           school_id|          teacher_id|project_type|project_grade_level_category|project_cost|project_posted_date|project_expiration_date|project_fully_funded_date|
+--------------------+--------------------+--------------------+------------+----------------------------+------------+-------------------+-----------------------+---------------------

## Donations&Projects

In [57]:
#combine donation and project information
joined_df = df_donations.join(df_projects, df_donations.project_id == df_projects.project_id, 'left').drop(df_projects.project_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
joined_df.printSchema()
joined_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- project_id: string (nullable = true)
 |-- donation_id: string (nullable = true)
 |-- donor_id: string (nullable = true)
 |-- donation_included_optional_donation: string (nullable = true)
 |-- donation_amount: double (nullable = true)
 |-- donation_received_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- school_id: string (nullable = true)
 |-- teacher_id: string (nullable = true)
 |-- project_type: string (nullable = true)
 |-- project_grade_level_category: string (nullable = true)
 |-- project_cost: double (nullable = true)
 |-- project_posted_date: date (nullable = true)
 |-- project_expiration_date: date (nullable = true)
 |-- project_fully_funded_date: date (nullable = true)

+--------------------+--------------------+--------------------+-----------------------------------+---------------+----------------------+----+--------------------+--------------------+------------+----------------------------+------------+-------------------+-----------------

In [59]:
joined_df.createOrReplaceTempView('donations')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
#data quality check
spark.sql('''select count(*) from donations''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|  435978|
+--------+

In [ ]:
joined_df.write.format("csv").save(output_data + 'donations')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…